# Visualize Notebook
In this notebook the Gold Dataframe will be read and extract information of it. The objective is to see the correlations between the variables and the GDP and also what countries have the highest correlation value.


## Imports
Start importing all the libraries and also the methods of pvalue and search indicators that will be used later in the notebook.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from scipy.stats import shapiro, pearsonr, spearmanr
import os
import statistics
import seaborn as sns
from scipy.stats import norm
import plotly.express as px
from IPython.display import display_html
import ipywidgets as widgets
from ipywidgets import Layout
from ipywidgets import AppLayout, Button, GridspecLayout
from ipywidgets import interact, interact_manual
from Project.Utils.visualize import  search


## Correlation dataframe.
This dataframe is the main piece of the notebook. Consists in generating for every country the correlation matrix for it and saving only the correlation value of the different variables with the GDP. 
This codeblock also calculates the p-value of Spearman and Pearson, if the value > 0.05 the correlation will be deleted due to not having statiscal significance.

Later on is concatenated and generates the following result:

In [2]:
#One dataframe per country

read_path = os.getcwd() + '/Output/'
write_path = os.getcwd() + '/Output/'

col_country = 'Country'
col_year = 'Year'
col_region = 'Region'
col_gdp = 'GDP'

df= pd.read_csv (read_path + 'GoldDataframe.csv')
corr_df = pd.DataFrame()
corr_df.index.names = [col_country]

#List all the countries, none repeated
countries = set(df[col_country].to_list())

country_dict = {}
corr_dict = {}

for country in countries:
    #Get the DataFrame for a given country
    country_df = df.loc[df[col_country] == country]

    #Correlation matrix for that country
    country_corr_df = country_df.corr()

    #Significance for the correlations
    pval = country_df.corr(method = lambda x, y: pearsonr(x, y)[1]) - np.eye(*country_corr_df.shape)
    p = pval.applymap(lambda x: 1 if x < 0.05 else np.NaN)
    country_corr_df = country_corr_df * p

    pval = country_df.corr(method = lambda x, y: spearmanr(x, y)[1]) - np.eye(*country_corr_df.shape)
    p = pval.applymap(lambda x: 1 if x < 0.05 else np.NaN)
    country_corr_df = country_corr_df * p

    #Trim it into a single row
    country_corr_df = country_corr_df.rename(columns = {col_gdp: country}).drop(index = [col_year, col_gdp])

    #Add the row to a new DataFrame with the correlations for each country
    corr_df = pd.concat([corr_df, country_corr_df[country]], axis = 1)

#Transpose the resulting DataFrame to have the desired format, save it and show it
corr_df = corr_df.transpose()
corr_df.to_csv(os.getcwd()+'/Output/Corr_DF.csv')
corr_df


c:\Users\smanoles\Documents\GitHub\python-data-driven-decisions\.venv\lib\site-packages\scipy\stats\_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
c:\Users\smanoles\Documents\GitHub\python-data-driven-decisions\.venv\lib\site-packages\scipy\stats\_stats_py.py:4529: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())
c:\Users\smanoles\Documents\GitHub\python-data-driven-decisions\.venv\lib\site-packages\scipy\stats\_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
c:\Users\smanoles\Documents\GitHub\python-data-driven-decisions\.venv\lib\site-packages\scipy\stats\_stats_py.py:4529: SpearmanRConstantInputWarning: An input array is constant; the correlati

,AgriShareGDP,CreditToAgriFishForest,EmploymentRural,TotalAgri,% Soldiers,Birth Rate,Death Rate,Homicides,Life Expectancy,Maternal Death Risk,...,% Population Growth,% Rural Population,Civil Liberties,Freedom of Expression,% Healthcare Investment,% Employment Industry,% Education Expenditure,% Men Employment,% Women Employment,Population
Egypt,-0.514386,NaN,NaN,0.922174,-0.823953,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.706273,NaN,-0.535786,0.720349,NaN,NaN,NaN,0.901123
Cuba,-0.741178,NaN,NaN,NaN,-0.683030,-0.964362,0.970615,NaN,0.942638,-0.697380,...,-0.528841,-0.958753,0.901891,0.682908,0.823339,-0.939949,NaN,NaN,NaN,0.879921
Libya,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.631893,...,-0.471225,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sudan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Mozambique,0.567766,-0.857628,NaN,0.887298,NaN,-0.866994,-0.815453,NaN,0.734361,-0.908427,...,NaN,-0.785036,NaN,NaN,-0.685869,0.744743,NaN,NaN,NaN,0.819696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Argentina,-0.503461,0.656928,NaN,0.950021,-0.696227,-0.715829,-0.800463,NaN,0.772551,-0.830826,...,NaN,-0.780874,NaN,-0.597334,NaN,0.726257,-0.640218,NaN,NaN,0.763368
Costa Rica,-0.983603,NaN,-0.600957,0.918202,-0.723935,-0.936195,0.986153,0.796409,0.981229,-0.864248,...,-0.938493,-0.979714,NaN,-0.800748,NaN,-0.938845,0.875543,-0.938355,NaN,0.981640
Belarus,-0.740688,NaN,NaN,0.969914,NaN,0.714361,-0.572392,-0.898360,0.758016,-0.919550,...,0.753786,-0.809223,NaN,NaN,NaN,-0.801186,NaN,NaN,NaN,-0.932844
Gambia,-0.504079,NaN,NaN,0.802629,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.592211,NaN,NaN,0.799523


## Cleaned GoldDataframe 
Before the correlation dataframe was generated and it detected all the correlations with a high p-value. To perform all the following analysis a clean GoldDataframe is needed. It simply detects if the correlation for the indicator and country is Nan and procedes to delete the whole column indicator for the country.

In [3]:

df = df.copy()
for country in countries:
    for ind in corr_df.columns:
        if np.isnan(corr_df[ind][country]):
            df.loc[df[col_country] == country, [ind]] = np.NaN

df.set_index([col_country, col_region, col_year]).to_csv(write_path + 'GoldDataframe_Clean.csv')
df.set_index([col_country, col_region, col_year])



AgriShareGDP  CreditToAgriFishForest  \
Country     Region             Year                                         
Afghanistan South Asia         2000      54.06300                     NaN   
                               2001      54.06300                     NaN   
                               2002      45.13440                     NaN   
                               2003      41.90340                     NaN   
                               2004      35.61280                     NaN   
...                                           ...                     ...   
Zimbabwe    Sub-Saharan Africa 2016       7.87399                     NaN   
                               2017       8.34095                     NaN   
                               2018       8.30469                     NaN   
                               2019       8.17322                     NaN   
                               2020      10.93630                     NaN   

                                     EmploymentRural           GDP  TotalAgri  \
Country     Region             Year                                             
Afghanistan South Asia         2000              NaN   3342.034168        NaN   
                               2001              NaN   3598.470576        NaN   
                               2002              NaN   4141.523943        NaN   
                               2003              NaN   4729.042179        NaN   
                               2004              NaN   5388.482107        NaN   
...                                              ...           ...        ...   
Zimbabwe    Sub-Saharan Africa 2016              NaN  20548.678073        NaN   
                               2017              NaN  22040.902301        NaN   
                               2018              NaN  24311.560545        NaN   
                               2019              NaN  21935.075306        NaN   
                               2020              NaN  21786.743637        NaN   

                                     % Soldiers  Birth Rate  Death Rate  \
Country     Region             Year                                       
Afghanistan South Asia         2000         NaN      48.021      11.718   
                               2001         NaN      47.505      11.387   
                               2002         NaN      46.901      11.048   
                               2003         NaN      46.231      10.704   
                               2004         NaN      45.507      10.356   
...                                         ...         ...         ...   
Zimbabwe    Sub-Saharan Africa 2016     0.75975         NaN       8.286   
                               2017     0.75072         NaN       8.044   
                               2018     0.73821         NaN       7.883   
                               2019     0.73821         NaN       7.773   
                               2020     0.73821         NaN       7.692   

                                     Homicides  Life Expectancy  ...  \
Country     Region             Year                              ...   
Afghanistan South Asia         2000        NaN           55.841  ...   
                               2001        NaN           56.308  ...   
                               2002        NaN           56.784  ...   
                               2003        NaN           57.271  ...   
                               2004        NaN           57.772  ...   
...                                        ...              ...  ...   
Zimbabwe    Sub-Saharan Africa 2016        NaN           60.294  ...   
                               2017        NaN           60.812  ...   
                               2018        NaN           61.195  ...   
                               2019        NaN           61.490  ...   
                               2020        NaN           61.738  ...   

                                     % Population Growth  % Rural Po

# Analysis
From now on all the data will be analysed with the Cleaned dataframes.

## Choropleth map
This block uses the IppyWidget for the indicator list and the Choroplet for the Map. 

In this section a map is generated and painted with the values of the indicator. You can salect the indicator from the list and an new map is generated, also along 2 lists of the highest and lowest countries for the selected indicator. 




In [4]:
indicator = widgets.SelectMultiple(
    options = corr_df.columns.tolist(),
    value = ['AgriShareGDP'],
    description='Indicator',
    disabled=False,
    layout = Layout(width='50%', height='80px')
)



def globalGrapgh(indicator):
    ind = indicator[0]
    N = 10
    fig = px.choropleth(corr_df, locations = corr_df.index, locationmode='country names', 
                        color= indicator[0],projection="natural earth",
                        color_continuous_scale='RdBu',
                        width=700, height=500)

    pos_corr = corr_df.drop(corr_df.columns.difference([ind]), axis = 1).sort_values(by = ind, axis = 0, ascending = False).head(n = N)
    neg_corr = corr_df.drop(corr_df.columns.difference([ind]), axis = 1).sort_values(by = ind, axis = 0, ascending = True).head(n = N)

    fig.update(layout_coloraxis_showscale=True)
    fig.show()
    

    pos_styler = pos_corr.style.set_table_attributes("style='display:inline'").set_caption('Direct correlation')
    neg_styler = neg_corr.style.set_table_attributes("style='display:inline'").set_caption('Inverse correlation')

    space = "\xa0" * 10
    display_html(pos_styler._repr_html_()+ space  + neg_styler._repr_html_(), raw=True)


widgets.interactive(globalGrapgh, indicator = indicator)

interactive(children=(SelectMultiple(description='Indicator', index=(0,), layout=Layout(height='80px', width='…

## Country Indicators
In this codeblock a widged is implmented to generate a table with the indicators. In order to make it work just select the country from the dropdown and procede to establasih the threshold. The default value for the threshold is 0.7, which we consider to be the minimum to consider an indicator correlated to the GDP. To analyze the results:
- H0: the indicator and the GDP are uncorrelated.
- H1: the indicator and the GDP are correlated.

The p-value represents the probability that your data would have arisen if the null hypothesis were true. If it's higher than 0.05 then it has no statistical signifcance. If an indicator is lower than 0.05 it has no significance and would be painted the cell in red.

If the both correlations are  >0.7 or <-0.7 and the p-values are < 0.05 then we have a high correlation that can be applied to the whole population.

The code consists in a IppyWidget with a slider and a dropdown. Once this parameters have been set it calls the method 'search' and applies a style format of the returned Dataframe.

In [5]:
def tableOut(Threshold, Country):

    df = search(Threshold, 'Country', Country)
    if df.empty:
        return print("No indicators have been found.")

    left = pd.Series([0.05, 0.05], index=['P-value Pearson', 'P-value Spearman'])
    left2 = pd.Series([-1, -1], index=['GDP Pearson Corr', 'GDP Spearman Corr'])
    left3 = pd.Series([0, 0], index=['GDP Pearson Corr', 'GDP Spearman Corr'])
    df =df.style.highlight_between(left=left, right=1.5, axis=1, props='color:white; background-color:red;')\
        .highlight_between(left=left2, right=1.5, axis=1, props='color:white; background-color:#929bfc;')\
        .highlight_between(left=left3, right=1.5, axis=1, props='color:white; background-color:#b3b9ff;')\
        .format('{:,.4f}', subset = ['GDP Pearson Corr', 'GDP Spearman Corr'])\
        .format('{:,.12f}', subset = ['P-value Pearson', 'P-value Spearman']) 
    
    display(df)

    

@interact(
    Country = sorted(corr_df.index.tolist()),
    Threshold = (0, 1, 0.05))
def g(Country = 'Afghanistan', Threshold = 0.7):
    return tableOut(Threshold,Country)

    
        

interactive(children=(Dropdown(description='Country', options=('Afghanistan', 'Albania', 'Algeria', 'Andorra',…

## Region Indicators
It's almost the same codeblock as before, but for the regions. The groups have been formed according to the Dataframe of World Data Bank.


In [6]:
def tableOut2(Threshold, Region):

    df = search(Threshold, 'Region', Region)
    if df.empty:
        return print("No indicators have been found.")

    left = pd.Series([0.05, 0.05], index=['P-value Pearson', 'P-value Spearman'])
    left2 = pd.Series([-1, -1], index=['GDP Pearson Corr', 'GDP Spearman Corr'])
    left3 = pd.Series([0, 0], index=['GDP Pearson Corr', 'GDP Spearman Corr'])
    df = df.style.highlight_between(left=left, right=1.5, axis=1, props='color:white; background-color:red;')\
        .highlight_between(left=left2, right=1.5, axis=1, props='color:white; background-color:#929bfc;')\
        .highlight_between(left=left3, right=1.5, axis=1, props='color:white; background-color:#b3b9ff;')\
        .format('{:,.4f}', subset = ['GDP Pearson Corr', 'GDP Spearman Corr'])\
        .format('{:,.12f}', subset = ['P-value Pearson', 'P-value Spearman']) 
    
    display(df)



@interact(
    Region = set(df['Region'].to_list()),
    Threshold = (0, 1, 0.05))
def g(Region = 'East Asia and Pacific', Threshold = 0.7):
    return tableOut2(Threshold, Region)

    

interactive(children=(Dropdown(description='Region', options=('East Asia and Pacific', 'South Asia', 'Middle E…

## Global Indicators
This section reperesents the final analysis of the indicators. It shows in the indicators table with the highest correlations. With this results we can give an answer to the hypothesis of the project and establish the indicators with a high GDP relation.

In [10]:
def tableOut3(Threshold):

    df = search(Threshold, 'Global')
    if df.empty:
        return print("No indicators have been found.")

    left = pd.Series([0.05, 0.05], index=['P-value Pearson', 'P-value Spearman'])
    left2 = pd.Series([-1, -1], index=['GDP Pearson Corr', 'GDP Spearman Corr'])
    left3 = pd.Series([0, 0], index=['GDP Pearson Corr', 'GDP Spearman Corr'])
    df =df.style.highlight_between(left=left, right=1.5, axis=1, props='color:white; background-color:red;')\
        .highlight_between(left=left2, right=1.5, axis=1, props='color:white; background-color:#929bfc;')\
        .highlight_between(left=left3, right=1.5, axis=1, props='color:white; background-color:#b3b9ff;')\
        .format('{:,.4f}', subset = ['GDP Pearson Corr', 'GDP Spearman Corr'])\
        .format('{:,.12f}', subset = ['P-value Pearson', 'P-value Spearman']) 
    
    display(df)



@interact(
    Threshold = (0, 1, 0.05))
def g(Threshold = 0.7):
    return tableOut3(Threshold)


interactive(children=(FloatSlider(value=0.7, description='Threshold', max=1.0, step=0.05), Output()), _dom_cla…

## Median Global Indicators
Reading the corr_df generated at the beggining of this notebook it calculates the median of each column. This approximation is very simple and might give an idea if the aggregation method is correct. Obviously the result won't be the same but the tendency has to be similar.

Having this in mind for each column in the corr_df import the name of the indicator and compute the median. Save this new dataframe in result_df.

In [8]:
result_df = pd.DataFrame()
for column in corr_df.columns:
    aux = pd.DataFrame({'Indicator': [column],
                        'GDP Pearson Corr': [corr_df[column].median()]})
    result_df = pd.concat([result_df, aux], ignore_index=False, axis = 0)
    result_df = result_df.sort_values(by=["GDP Pearson Corr"], ascending = False)
    

result_df.set_index(['Indicator'], inplace=True)

Using a IppyWidget display the previously generated result_df. Lowering the Threshold more than 0.7 will show indicators that don't have a real correlation with the GDP, procede with caution extracting conclusions of them.

In [9]:
def tableOutMedian(Threshold):
    df = pd.concat([result_df.loc[result_df['GDP Pearson Corr'] >= Threshold], result_df.loc[result_df['GDP Pearson Corr'] <= -Threshold]], axis = 0)

    left2 = pd.Series([-1], index=['GDP Pearson Corr'])
    left3 = pd.Series([0], index=['GDP Pearson Corr'])
    df = df.style.highlight_between(left=left2, right=1.5, axis=1, props='color:white; background-color:#929bfc;')\
        .highlight_between(left=left3, right=1.5, axis=1, props='color:white; background-color:#b3b9ff;')\
        .format('{:,.4f}', subset = ['GDP Pearson Corr'])
    display(df)



@interact(
    Threshold = (0, 1, 0.05))
def g(Threshold = 0.7):
    return tableOutMedian(Threshold)

interactive(children=(FloatSlider(value=0.7, description='Threshold', max=1.0, step=0.05), Output()), _dom_cla…